# Atmospherically Corrected Earth Engine Time Series 

### Overview

This notebook creates atmospherically corrected time series of satellite imagery using [Google Earth Engine](https://earthengine.google.com/) and the [6S emulator](https://github.com/samsammurphy/6S_emulator). 

### Supported missions

* Sentintel2
* Landsat8
* Landsat7
* Landsat5
* Landsat4

### Output

Average (cloud-free) pixel values.

#### Initialize

In [ ]:
import ee
ee.Initialize()

In [ ]:
import os
import sys
import numpy as np
from matplotlib import pylab as plt
import pandas as pd
%matplotlib inline
base_dir = os.path.dirname(os.getcwd())
sys.path.append(os.path.join(base_dir,'atmcorr'))

from ee_requests import request_meanRadiance
from atmcorr_timeseries import surface_reflectance_timeseries
from plots import plot_timeseries
from IPython.display import display, Image

### User Input

In [ ]:
# region of interest
geom = ee.Geometry.Point(-122.2586, 37.4967)

# start and end of time series
startDate = ee.Date('1980-01-01')# YYYY-MM-DD
stopDate  = ee.Date('2020-01-01')# YYYY-MM-DD

# satellite mission
mission = 'Sentinel2' # options = 'Sentinel2', 'Landsat8', 'Landsat7', 'Landsat5', 'Landsat4'

# cloud and cloud shadow mask?
removeClouds = True

#### Interpolated look-up tables

The emulator uses interpolated look-up tables to generate atmospheric correction coeffients.

In [ ]:
import interpolated_lookup_tables as iLUT

iLUTs = iLUT.handler(mission) 
iLUTs.get()

#### Mean Average Radiance

In [ ]:
# earth engine request
request = request_meanRadiance(geom, startDate, stopDate, mission, removeClouds = removeClouds) # True or False for now

In [ ]:
# get data locally (i.e. client side)
print('Getting data from Earth Engine.. ')
meanRadiance = request.getInfo()
print('Data collection complete')

#### Two or more images?

In [ ]:
num = len(meanRadiance['features'])
print('length of data collection = ',num)
if num < 2:
    print('*'*42+'\nneed at least 2 images, try user inputs?\n'+'*'*42)

#### Atmospheric Correction

In [ ]:
timeSeries = surface_reflectance_timeseries(meanRadiance, iLUTs, mission)

### Plots

#### Visible wavelengths

In [ ]:
plot_timeseries(timeSeries, 'visible')

#### Near infrared

In [ ]:
plot_timeseries(timeSeries, 'nir')

#### Short-wave Infrared

In [ ]:
plot_timeseries(timeSeries, 'swir')

#### Atmospheric bands

In [ ]:
atmcorr_bands_exist = plot_timeseries(timeSeries, 'atmospheric')
if not atmcorr_bands_exist:
    print('atmospheric bands not available for:',timeSeries['mission'])

#### Save to excel

In [ ]:
# create an excel directory
excel_dir = os.path.join(base_dir,'files','excel')
if not os.path.exists(excel_dir):
    os.makedirs(excel_dir)

# create pandas data frame
df = pd.DataFrame.from_dict(timeSeries)

# save to excel
df.to_excel(os.path.join(excel_dir, mission+'.xlsx'), index=False)